### Export HKW Metadata + FC To Shp

In [1]:
# Import arcpy module
import arcpy
import numpy
import os
import pandas as pd
from IPython.display import display, HTML
from arcpy import metadata as md

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


### 1) Input Shapefles

- Innovataion

In [2]:
list_shp = [r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0",
            # r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_Dynamic_Cable_UTM31_v0",
            #r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\GRID\GRID.gdb\NL_IJV_Beta_EAC_Solar_Dynamic_Cable_L007_250D_DA6_58T_loops_UTM31N_v0"          
                        
            ]


folder = r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_Innovation"

excel_file = r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\5_PROJECTDOCS\NL_IJV_Dataset_Inventory.xlsx"


- 4 sites

In [5]:
list_shp = [r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\WTG\WTG.gdb\NL_IJV_I_WTG_L005_276D_DA6_50T_UTM31N_v1",
            r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\WTG\WTG.gdb\NL_IJV_II_WTG_L005_276D_DA6_50T_UTM31N_v1",
            r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\WTG\WTG.gdb\NL_IJV_III_WTG_L005_276D_DA6_50T_UTM31N_v1",
            r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\WTG\WTG.gdb\NL_IJV_IV_WTG_L005_276D_DA6_50T_UTM31N_v1"          
            
            
            ]


folder = r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts"

### 2) Metadata from excel to GDB

In [17]:

# String to search for
df = pd.read_excel(excel_file)

for i in list_shp:
    if arcpy.Exists(i):
        
        search_string = i.split(os.path.sep)[-1]
        filtered_df = df[df["Full Name"] == search_string]
        print('Input file:',search_string )

        if not filtered_df.empty:
            # Get the text from text_column in the same row
            imported_title = filtered_df.iloc[0]["Full Name"]
            imported_summary = filtered_df.iloc[0]["Summary"]
            imported_tags = filtered_df.iloc[0]["Tags"]
            imported_Description = filtered_df.iloc[0]["Description"]
            imported_Credits = filtered_df.iloc[0]["Credits"]
            imported_Date = str(filtered_df.iloc[0]["Date Created"])

            imported_Description = imported_Description + "\n" + imported_Date

            # Print or use the imported_text as needed
            print(" - imported_title:", imported_title)
            print(" - imported_Date:", imported_Date)

        else:
            print("Search string not found in the specified column!!")

        # Create a new Metadata object and add some content to it
        new_md = md.Metadata()
        new_md.title = imported_title
        new_md.tags = imported_tags
        new_md.summary = imported_summary
        new_md.description = imported_Description
        new_md.credits = imported_Credits

        # Assign the Metadata object's content to a target item
        tgt_item_md = md.Metadata(i)
        if not tgt_item_md.isReadOnly:
            tgt_item_md.copy(new_md)
            tgt_item_md.save()


Input file: NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0
 - imported_title: NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0
 - imported_Date: 2023-10-19 00:00:00
Input file: NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_Dynamic_Cable_UTM31_v0
 - imported_title: NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_Dynamic_Cable_UTM31_v0
 - imported_Date: 2023-10-19 00:00:00
Input file: NL_IJV_Beta_EAC_Solar_Dynamic_Cable_L007_250D_DA6_58T_loops_UTM31N_v0
 - imported_title: NL_IJV_Beta_EAC_Solar_Dynamic_Cable_L007_250D_DA6_58T_loops_UTM31N_v0
 - imported_Date: 2023-10-19 00:00:00


### 3) Your New Alias Name

In [22]:
for i in list_shp:
    if arcpy.Exists(i):
        desc = arcpy.Describe(i)
        old_alias = desc.aliasName
        fc = i.split(os.path.sep)[-1]
        new_alias = fc.replace("_", " ")

    try:
        # Set the new alias name for the feature class
        arcpy.AlterAliasName(i, new_alias)
        print(f"Alias'{old_alias}' to '{new_alias}' successfully.")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Alias'NL IJV Solar Buffer 450m L007 GE250D DA6 58T UTM31 v0' to 'NL IJV Solar Buffer 450m L007 GE250D DA6 58T UTM31 v0' successfully.
Alias'NL IJV Solar Buffer 450m L007 GE250D DA6 58T Dynamic Cable UTM31 v0' to 'NL IJV Solar Buffer 450m L007 GE250D DA6 58T Dynamic Cable UTM31 v0' successfully.
Alias'NL IJV Beta EAC Solar Dynamic Cable L007 250D DA6 58T loops UTM31N v0' to 'NL IJV Beta EAC Solar Dynamic Cable L007 250D DA6 58T loops UTM31N v0' successfully.


2) Export FC to SHP

In [25]:
for i in list_shp:
    print('Input file:', i)
    fc_name = i.split(os.path.sep)[-1]

    if fc_name:
        name_shp = fc_name + ".shp"
        output_shp = os.path.join(folder, fc_name, fc_name+ ".shp")
        folder_shp = os.path.join(folder, fc_name)
        if not os.path.exists(folder_shp):
            os.makedirs(folder_shp)
        if not os.path.exists(output_shp):
            arcpy.FeatureClassToFeatureClass_conversion(i, folder_shp, name_shp)
            print(f"- Shapefile '{output_shp}' created successfully.")


Input file: \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0
- Shapefile '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_Innovation\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0.shp' created successfully.
Input file: \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_Dynamic_Cable_UTM31_v0
- Shapefile '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_Innovation\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_Dynamic_Cable_UTM31_v0\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_Dynamic_Cable_UTM31_v0.shp' created successfully.
Input file: \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\GRID\GRID.gdb\NL_IJV_Beta_EAC_Solar_Dynamic_Cable_L007_250D_DA6_58T_loops_UTM31N_v0
- Shapefile '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_Innovation\NL_IJV_Beta_EAC_Solar_Dynamic_Cable_L007_250D_DA6_58T_loops_UTM31N

### 3) calculate the centroid of polygons (Buffer) to Attri

In [3]:
# Spatial reference (Coordinate system) - 25831 in this case
sr = arcpy.SpatialReference(25831)

# Create a list to store centroid coordinates
centroid_coords_list = []

list_shp = [r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0",
            
            ]


# Iterate through the polygons and calculate centroids
for i in list_shp:
    print('Input file:', i)
    field_list = [field.name for field in arcpy.ListFields(i)]

    del_fields = ["Centroid_X", "Centroid_Y","X_ETRS_UTM31", "Y_ETRS_UTM31"]
    arcpy.DeleteField_management(i, del_fields)

    if not "Centroid_X" in field_list:
        # Rename the fields POINT_X and POINT_Y to X_COORD and Y_COORD
        arcpy.AddField_management(i, "Centroid_X", "DOUBLE")
        arcpy.AddField_management(i, "Centroid_Y", "DOUBLE")

        # Rename the fields POINT_X and POINT_Y to X_COORD and Y_COORD
        arcpy.AlterField_management(i, "Centroid_X", "X_ETRS_UTM31", "Easting [ETRS 1989 UTM31] ")
        arcpy.AlterField_management(i, "Centroid_Y", "Y_ETRS_UTM31", "Northing [ETRS 1989 UTM31]")

    fc_name = i.split(os.path.sep)[-1]

    # Iterate through the polygons and calculate centroids
    with arcpy.da.SearchCursor(i, ["SHAPE@", "SHAPE@XY"]) as cursor:
        for row in cursor:
            polygon_geometry = row[0]
            centroid_x, centroid_y = row[1]
            

            # Add coordinates to the list
            centroid_coords_list.append(((round(centroid_x,1)),round(centroid_y,1)))
                                        
    print(centroid_coords_list)
    # Update the attribute table with centroid coordinates
    with arcpy.da.UpdateCursor(i, ["SHAPE@", "X_ETRS_UTM31", "Y_ETRS_UTM31"]) as update_cursor:
        for row, centroid_coords in zip(update_cursor, centroid_coords_list):
            row[1] = centroid_coords[0]
            row[2] = centroid_coords[1]
            update_cursor.updateRow(row)


Input file: \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0


ExecuteError: ERROR 000464: Cannot get exclusive schema lock.  Either being edited or in use by another application or service.
Failed to execute (DeleteField).


3) Export FC to Excel

In [4]:
for i in list_shp:
    print('Input file:', i)
    fc_name = i.split(os.path.sep)[-1]

    if fc_name:
    
        output_excel = os.path.join(folder, fc_name, fc_name + ".xlsx")
        folder_excel = os.path.join(folder, fc_name)

        if not os.path.exists(folder_excel):
            os.makedirs(folder_excel)
            # Export attribute table to Excel
        if not os.path.exists(output_excel):
            arcpy.TableToExcel_conversion(i, output_excel)
            print(f"- Excel '{output_excel}' created successfully.")


Input file: \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0
- Excel '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_Innovation\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0.xlsx' created successfully.


4) Export FC to DWG

In [56]:
for i in list_shp:
    print('Input file:', i)
    fc_name = i.split(os.path.sep)[-1]

    if fc_name:
        # Output DWG file
        dwg_folder = os.path.join(folder, fc_name + '_DWG')
        dwg_output = os.path.join(dwg_folder, fc_name +'.dwg')
        if not os.path.exists(dwg_folder):
            os.makedirs(dwg_folder)
        if not os.path.exists(dwg_output):
            arcpy.conversion.ExportCAD(output_shp, "DWG_R2018", dwg_output, False, False)
            print(f"- DWG '{dwg_folder}' created successfully.")


Input file: \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\INNOVATION\INNOVATION.gdb\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0
- DWG '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_Innovation\NL_IJV_Solar_Buffer_450m_L007_GE250D_DA6_58T_UTM31_v0_DWG' created successfully.
